In [ ]:
#import modules
#Initialize packages
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
os.chdir("C:\\Users\\apate\\OneDrive - Northeastern University\\Dennis Lab site\\AP\\Spectral_demixing\\notebooks\\PIPELINE") #change directory to access PIPELINE modules
import Step3_demixing as dm
from skimage import measure
from skimage.morphology import disk
from skimage.filters import threshold_otsu
from scipy import ndimage
from pathlib import Path
import pickle

In [ ]:
os.chdir("C:\\Users\\apate\\OneDrive - Northeastern University\\Dennis Lab site\\BU Research Drive Data\\Amish\Image processing\\Coding\\NIR-I_spectral_demixing")
#Load input data and number of depths imaged. Input file should be a 3D numpy array with filter channels in the last dimension
inputData = ".\\Results\\Liquid\\240508_backSub_purespectra\\liquidPhantomStackSlice.npy"
depth = []

#Reference spectra value input. References obtained from ROIs of 0mm depth liquid phantom. ROIs located in raw images folder. Chan1 is QD800, Chan2 is QD705
#reference line: [0,38,42,46,50,54,58,62,100]
df_ref = pd.DataFrame(data={'Chan 1':[3222513.708, 12002950.53, 47117561.32, 188029704.3, 601431934.2, 1476311938.0, 2729065829.0, 3046208205.0, 3235180670.0, 2767274415.0],
                            'Chan 2':[293811595.7, 1331699182.0, 2519761632.0, 2521315550.0, 1956998013.0, 1239274007.0, 763586839.8, 387347557.2, 230544085.4, 134733326.2]},
                      index=['BP660', 'BP680', 'BP700', 'BP720', 'BP740', 'BP760','BP780','BP800','BP820','BP840'])


specMethod = "PURE SPECTRA" # "SIMPLISMA" or "PURE SPECTRA"
backSubOpt = "Ball" # "Threshold" or "Ball"
waterSpec = False  #If using water spectra, set to True


In [ ]:
#Collect number of channels, filters, and shape of input data
rawData = np.load(inputData)
Xshape = np.shape(rawData)
nchan = np.shape(df_ref)[1]
nfilt = np.shape(df_ref)[0]
ndepth = len(depth)

#Set working directory
here = Path(inputData).resolve().parent
os.chdir(here)
os.chmod(here, 0o755)


In [ ]:
#Plot raw images summed up over all channels
rawSum = np.zeros([Xshape[0],Xshape[1],ndepth])
for i in range(0,ndepth):
    rawSum[...,i] = np.sum(rawData[...,(i*nfilt):(i*nfilt+nfilt)],axis = 2)
fig_raw, axs5 = plt.subplots(1,ndepth, figsize=(2*ndepth,4))
for i in range(0,ndepth):
        make_image(axs5[i], rawSum[:,:,i], [Xshape[0],Xshape[1]])

In [ ]:
#Fit MCR-ALS on input data
mcr_out, mcr_raw_flat, mcr_raw, mcr_conc, mcr_image,St_fit,St_fit_nonConstrained = dm.fit_mcrals(inputData, df_ref.T,ndepth,nfilt,nchan, specMethod, waterSpec) #Ensure reference spectra is in an array-like structure (ndarray)

In [ ]:
# Convert the weights to an WxL image

mcr_image = np.reshape(mcr_conc, [Xshape[0],Xshape[1],nchan,ndepth])   

sfig, axs = plt.subplots(nchan,ndepth, figsize=(2*ndepth,4))
#mcr_conc[:,1,9] = np.zeros([Xshape[0]*Xshape[1]]) #Test for poisoning mcr_conc data set during plotting

for i in range(0,ndepth):
    for j in range(0,nchan):
        #make_image(axs[j,i], mcr_conc[:,j,i], [Xshape[0],Xshape[1]])
        axs[j,i].imshow(mcr_image[:,:,j,i], cmap='gray')
        

In [ ]:
#Export MCR fits to file
np.save('mcr_conc.npy', mcr_conc)
np.save('mcr_backSub.npy', backSub)
np.save('mcr_image.npy', mcr_image)
np.save('mcr_raw.npy', rawData)
np.save('backgroundMask.npy', backgroundMask)
np.save('St_fit_nonConstrained.npy', St_fit_nonConstrained)

In [ ]:
#Load input data and number of depths imaged. Input file should be a 3D numpy array with filter channels in the last dimension
os.chdir("C:\\Users\\apate\\OneDrive - Northeastern University\\Dennis Lab site\\BU Research Drive Data\\Amish\\Image processing\\Coding\\NIR-I_spectral_demixing")

fileName = ".\\Results\\Liquid\\240508_backSub_purespectra\\mcr_image.npy"

depth = [0,0.2,0.4,0.6,0.8,1,1.2,1.4,1.6,1.8,2]
filter = 0

#Reference spectra value input. References obtained from ROIs of 0mm depth liquid phantom. ROIs located in raw images folder. Chan1 is QD800, Chan2 is QD705
#reference line: [0,38,42,46,50,54,58,62,100]
df_ref = pd.DataFrame(data={'Chan 1':[3222513.708, 12002950.53, 47117561.32, 188029704.3, 601431934.2, 1476311938.0, 2729065829.0, 3046208205.0, 3235180670.0, 2767274415.0],
                            'Chan 2':[293811595.7, 1331699182.0, 2519761632.0, 2521315550.0, 1956998013.0, 1239274007.0, 763586839.8, 387347557.2, 230544085.4, 134733326.2]},
                      index=['BP660', 'BP680', 'BP700', 'BP720', 'BP740', 'BP760','BP780','BP800','BP820','BP840'])

#Extract number of channels and number of depths
mcr_image = np.array(np.load(fileName))
nchan = mcr_image.shape[2]
ndepth = mcr_image.shape[3]

#Set working directory
here = Path(fileName).resolve().parent
os.chdir(here)
os.chmod(here, 0o755)

#Load other input data
#mcr_raw = np.load(".\\mcr_raw.npy")

In [ ]:
#Create Masks of MCR-ALS output. 
#Initial thresholding to make binary image clear
#Convert image to binary b/w and fill in holes
#Find blobs and store them in a structure

filled_masks, backMask, labeledObjects, objectData= binarize_and_measure(mcr_image, 0.15,5,depth)  #Give masks at each depth for which to calculate CNR

sfig, axs = plt.subplots(1,ndepth, figsize=(2*ndepth,4))
for i in range(0,ndepth):
    #make_image(axs[j,i], mcr_conc[:,j,i], [Xshape[0],Xshape[1]])
    axs[i].imshow(filled_masks[:,:,i], cmap='gray')
